<a href="https://colab.research.google.com/github/parthag1201/DocReversalEngine/blob/main/FS_TS_Crew_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages for crew and langchain
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_google_genai

In [3]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [4]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Import crewAI library
from crewai import Agent, Task, Crew, Process

In [6]:
import os
from google.colab import userdata

In [7]:
abap_code_string = """
REPORT Z_MCCAIN_SALES_ANALYSIS.

TABLES: vbak, vbap, mara.

DATA: gt_sales_data TYPE TABLE OF ty_sales,
      gs_sales_data TYPE ty_sales.

*----------------------------------------------------------------------*
* Selection Screen
*----------------------------------------------------------------------*
SELECT-OPTIONS: s_vbeln FOR vbak-vbeln,
                s_matnr FOR mara.matnr.
PARAMETERS:     p_top TYPE i DEFAULT 10.

*----------------------------------------------------------------------*
* Start of Selection
*----------------------------------------------------------------------*
START-OF-SELECTION.
  PERFORM get_sales_data.
  PERFORM display_sales_data.

*&---------------------------------------------------------------------*
*&      Form  GET_SALES_DATA
*&---------------------------------------------------------------------*
FORM get_sales_data.
  SELECT vbeln, posnr, matnr, netwr
    FROM vbap
    INTO CORRESPONDING FIELDS OF TABLE gt_sales_data
    WHERE vbeln IN s_vbeln AND matnr IN s_matnr
    UP TO p_top ROWS.
ENDFORM.

*&---------------------------------------------------------------------*
*&      Form  DISPLAY_SALES_DATA
*&---------------------------------------------------------------------*
FORM display_sales_data.
  LOOP AT gt_sales_data INTO gs_sales_data.
    WRITE: / gs_sales_data-vbeln,
             gs_sales_data-posnr,
             gs_sales_data-matnr,
             gs_sales_data-netwr.
  ENDLOOP.
ENDFORM.
"""

In [8]:
template_TS = """Table of Contents

1. Enhancement Narrative

1.1 Request Details

1.2 Enhancement Definition

2. Enhancement Functional Requirements

2.1 Business Requirements

2.1.1 Objective

2.1.2 Scope

2.2 As-Is/To-Be Functionality

2.2.1 Special Requirements

2.3 SAP Standard Reference

2.4 Inputs and Dependencies

2.5 Regulatory / Compliance Requirements

2.6 Assumptions

3. Enhancement Specifications

3.1 User Exit /BADI/Routines/ Details

3.2 Transaction Screens

3.2.1 Screen Flow

3.3 Enhancement Processing Logic – Key steps

3.4 Table Data Access

4. Error Handling

5. Level 0 Test Requirements

6. Additional Information

7. Acronyms, Abbreviations, and Terms

8. References

9. Document Attributes"""

In [9]:
template_FS = """
1. Enhancement Narrative

1.1 Request Details

1.2 Enhancement Definition

2. Enhancement Functional Requirements

2.1 Business Requirements

2.1.1 Objective

2.1.2 Scope

2.2 As-Is/To-Be Functionality

2.2.1 Special Requirements

2.3 SAP Standard Reference

2.4 Inputs and Dependencies

2.5 Regulatory / Compliance Requirements

2.6 Assumptions

3. Enhancement Specifications

3.1 User Exit /BADI/Routines/ Details

3.2 Transaction Screens

3.2.1 Screen Flow

3.3 Enhancement Processing Logic – Key steps

3.4 Table Data Access

4. Error Handling

5. Level 0 Test Requirements

6. Additional Information

7. Acronyms, Abbreviations, and Terms

8. References

9. Document Attributes
"""

In [10]:
# Agents
from langchain_community.llms import HuggingFaceHub # Import necessary class if using Hugging Face models
from langchain_google_genai import ChatGoogleGenerativeAI # Import the correct class for Gemini

# Initialize the Gemini LLM
os.environ['GOOGLE_API_KEY']=userdata.get('gemini_pro')
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")


abap_code_analyst = Agent(
    role="Senior ABAP Code Analyst",
    goal="Dissect the provided ABAP report code to extract all technical details and the logic flow of the report."
    "Focus on identifying data sources, selection screens, core logic, and output structures."
    "You need to go through each and every section of report to the extent form where a good Functional and Technical specification can be derived out of it"
    "This is the ABAP report code : '{abap_report_code}'",
    backstory="""You are an expert ABAP developer with decades of experience. You have an exceptional eye for detail
    and can instantly understand the flow and structure of any ABAP program. Your task is to analyze the
    code and provide a structured, raw analysis for the documentation team.""",
    allow_delegation=False,
    verbose=True,
    llm=gemini_llm,
)

functional_spec_drafter = Agent(
    role="Business Systems Analyst",
    goal="Translate the technical analysis of an ABAP report into a clear and concise"
   "functional specification document for business stakeholders."
   "Refer to this professional template for creating a business functional specification document. '{template_FS}'",
    backstory="""You are a bridge between the technical team and the business. You excel at taking complex
    technical jargon and reframing it in terms of business processes and objectives. Your functional
    specifications are legendary for their clarity and business relevance, enabling stakeholders to
    understand exactly what a report does.""",
    allow_delegation=True,
    verbose=True,
    llm=gemini_llm,
)

technical_spec_writer = Agent(
    role="Technical Documentation Specialist",
    goal="""Create a comprehensive and detailed technical specification document based on the
    ABAP code analysis."""
    "Refer to this professional template for creating a business technical specification document. '{template_TS}'",
    backstory="""You are a meticulous technical writer who specializes in creating documentation for developers.
    Your specifications are precise, well-structured, and follow industry best practices. Your work ensures
    that any developer can pick up the report for maintenance or enhancement with complete clarity
    on its internal workings.""",
    allow_delegation=True,
    verbose=True,
    llm=gemini_llm,
)

manager_agent = Agent(
    role='Manager Agent',
    goal='Oversee the generation and refinement of Functional and Technical Specifications, facilitate collaboration, incorporate reviewer feedback, and ensure final output meets all requirements.'
    'The FS and TS reports must contain all the important information extracted from the report fror the business team to understand each and every ascpet of the report easily',
    backstory="""You are an experienced SAP S4 project manager with a deep understanding of software development life cycles and well versed with the good practises of SAP, Busness needs and ABAP Development.
    You excel at breaking down complex tasks, coordinating teams, incorporating feedback, and ensuring quality deliverables.
    You are adept at communicating with both technical and business stakeholders.""",
    verbose=True,
    allow_delegation=True,
    llm=gemini_llm,
)

output_reviewer = Agent(
    role='Output Reviewer (Business Knowledge)',
    goal='Review consolidated Functional and Technical Specifications and provide constructive feedback to the Manager Agent for refinement, focusing on business alignment and completeness.',
    backstory="""You are a business stakeholder with a keen eye for detail and a deep understanding of the organizational goals and user needs.
    You critically evaluate specifications, ensuring they meet intended business objectives and provide actionable feedback for improvement.""",
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm,
)

In [11]:
# Tasks

analyze_code_task = Task(
    description="""Analyze the following ABAP report code: '{abap_report_code}'.
                   Your analysis must cover these key areas:
                   1.  **Data Sources:** Identify all database tables, views, or structures being used.
                   2.  **Selection Screen:** Detail all SELECT-OPTIONS and PARAMETERS, including their technical IDs and associated data elements.
                   3.  **Core Processing Logic:** Summarize the main logic blocks, including loops, conditionals (IF/CASE statements), and subroutine calls (PERFORM).
                   4.  **Data Output:** Describe how the final data is presented (e.g., ALV grid, classical report WRITE statements) and list the fields being displayed.""",
    expected_output="""A structured text document containing a raw, point-by-point technical breakdown of the ABAP report.
                       This output should be purely technical and serve as the foundational information for other agents.""",
    agent=abap_code_analyst,
)

# Task for the Functional Spec Drafter
functional_spec_task = Task(
    description="""Using the technical analysis provided by the ABAP Code Analyst, create a functional specification document.
                   The document should explain the report's purpose and functionality from a business user's perspective.
                   It must include:
                   1.  **Report Purpose:** A high-level summary of what the report achieves.
                   2.  **Selection Criteria:** Explain the input fields in simple business terms (e.g., "User can filter by Sales Document Number").
                   3.  **Processing Logic Summary:** Describe what the report does with the data in plain language (e.g., "The report calculates the total net value for the selected sales orders").
                   4.  **Output Description:** Describe the layout and columns of the final report from a user's point of view.""",
    expected_output="""A well-formatted functional specification document in Markdown. The language should be clear,
                       non-technical, and focused on the business value and utility of the report.""",
    agent=functional_spec_drafter,
)

# Task for the Technical Spec Writer
technical_spec_task = Task(
    description="""Using the detailed technical analysis from the ABAP Code Analyst, create a formal technical specification document.
                   This document must be structured for a developer audience and include:
                   1.  **Program Details:** Program ID, Title, and a brief technical overview.
                   2.  **Data Declarations:** List of key tables, internal tables, and complex variables.
                   3.  **Selection Screen Objects:** A table listing each selection screen field with its technical name and properties.
                   4.  **Detailed Logic Flow:** A step-by-step description of the program's execution logic, referencing specific subroutines and key logic blocks.
                   5.  **Output Details:** A technical breakdown of the output display, including the field catalog for ALV grids or the format of WRITE statements.""",
    expected_output="""A comprehensive technical specification document in Markdown. It should be precise, detailed, and
                       formatted professionally to serve as official technical documentation.""",
    agent=technical_spec_writer,
)


# Manager Task for initial consolidation
initial_consolidation_task = Task(
    description="""Consolidate the initial Functional and Technical Specifications into a single, cohesive document.
    Ensure both specifications are present and well-formatted for review.
    """,
    expected_output="A single Markdown document containing the initial Functional Specification and Technical Specification, clearly separated.",
    agent=manager_agent,
    context=[functional_spec_task, technical_spec_task],
    llm=gemini_llm,
)

# Output Reviewer Task to provide feedback to the manager
output_review_feedback_task = Task(
    description="""Review the provided consolidated Functional and Technical Specifications.
    Focus on business alignment, clarity, completeness, and accuracy from a business perspective.
    Provide constructive feedback to the Manager Agent, detailing any required changes or improvements.
    The output should be a clear, concise, and structured list of actionable feedback points, not a final report.
    """,
    expected_output="A structured list of feedback points (e.g., bullet points, numbered list) for the Manager Agent to use for refining the specifications. Example: '- Section X needs more detail on Y. - Clarify business impact of Z. - Ensure consistency between FS and TS for ABC.'",
    agent=output_reviewer,
    context=[initial_consolidation_task], # Reviewer reviews the initial consolidated output
    llm=gemini_llm,
)

# Manager Task for final refinement based on feedback
final_specification_task = Task(
    description="""Based on the initial Functional and Technical Specifications and the feedback provided by the Output Reviewer,
    make necessary revisions to both specifications. Your goal is to produce the final, polished, and
    approved Functional and Technical Specifications in a single consolidated document.
    The output must contain the actual Functional Specification and Technical Specification,
    incorporating all valid feedback. Ensure clear separation between the FS and TS sections in the final output.
    """,
    expected_output="A single, comprehensive Markdown document containing the final, refined Functional Specification and Technical Specification, clearly separated (e.g., using distinct headings like '# Functional Specification' and '# Technical Specification') and incorporating all reviewer feedback.",
    agent=manager_agent,
    context=[initial_consolidation_task, output_review_feedback_task], # Manager uses both initial specs and feedback
    llm=gemini_llm,
    output_file='final_consolidated_specifications.md' # Final output will be saved here
)

In [12]:
# Call the crew

abap_doc_crew = Crew(
    agents=[
        abap_code_analyst,
        functional_spec_drafter,
        technical_spec_writer,
        manager_agent,
        output_reviewer
    ],
    tasks=[
        analyze_code_task,
        functional_spec_task,
        technical_spec_task,
        initial_consolidation_task,
        output_review_feedback_task,
        final_specification_task # This task produces the final output after review
    ],
    process=Process.sequential,
    verbose=2
)

In [13]:
# Start execution

result = abap_doc_crew.kickoff(
    inputs={
        'abap_report_code': abap_code_string,
        'template_FS' : template_FS,
        'template_TS' : template_TS
        })

 [DEBUG]: == Working Agent: Senior ABAP Code Analyst
 [INFO]: == Starting Task: Analyze the following ABAP report code: '
REPORT Z_MCCAIN_SALES_ANALYSIS.

TABLES: vbak, vbap, mara.

DATA: gt_sales_data TYPE TABLE OF ty_sales,
      gs_sales_data TYPE ty_sales.

*----------------------------------------------------------------------*
* Selection Screen
*----------------------------------------------------------------------*
SELECT-OPTIONS: s_vbeln FOR vbak-vbeln,
                s_matnr FOR mara.matnr.
PARAMETERS:     p_top TYPE i DEFAULT 10.

*----------------------------------------------------------------------*
* Start of Selection
*----------------------------------------------------------------------*
START-OF-SELECTION.
  PERFORM get_sales_data.
  PERFORM display_sales_data.

*&---------------------------------------------------------------------*
*&      Form  GET_SALES_DATA
*&---------------------------------------------------------------------*
FORM get_sales_data.
  SELECT vb

In [14]:
from IPython.display import Markdown
Markdown(result)

# **Consolidated Final Specification: Sales Analysis Report (Z_MCCAIN_SALES_ANALYSIS)**

---

# **Part 1: Functional Specification**

---

### **1. Enhancement Narrative**

#### **1.1 Request Details**
This document provides the functional specification for an **enhancement** to the existing custom ABAP report, `Z_MCCAIN_SALES_ANALYSIS`. The purpose is to define the changes required to transform the report from a simple data retrieval tool into a meaningful sales analysis instrument, providing stakeholders with improved decision-making capabilities.

#### **1.2 Enhancement Definition**
The Sales Analysis Report is a tool designed to provide a quick and simple view of sales document items. The enhancement will modify the report to sort sales data by value, ensuring that users can easily identify the highest-value line items based on their selection criteria.

---

### **2. Enhancement Functional Requirements**

#### **2.1 Business Requirements**

##### **2.1.1 Objective**
As a sales manager, I want to see the top N highest-value line items for a specific material to quickly identify major sales and support tactical decision-making.

##### **2.1.2 Scope**
*   **In-Scope:** The report queries and displays data exclusively from the Sales Document Item level (i.e., individual line items within a sales order). Filtering is available for Sales Document Number and Material Number. The output will be sorted by Net Value in descending order.
*   **Out-of-Scope:** This report does not access or display sales document header information (e.g., customer details, overall order status) or material master data beyond the number itself. The report is for display only and does not perform any data updates.

#### **2.2 As-Is/To-Be Functionality**
This section describes the current functionality of the report and the future-state functionality of the proposed enhancement.

**As-Is (Current Functionality):**
The report currently functions as a basic query tool. A user provides filter criteria, and the report returns a list of sales items that match. A key feature is the 'Maximum Records to Display' parameter, which allows the user to limit the output size.

**Critical Limitation:** The 'Maximum Records to Display' feature currently returns an arbitrary set of records. For example, if a user requests the 'top 10' records, the system retrieves the first 10 matching items it finds in the database without any specific sorting logic. The results are **not** sorted by value, date, or any other business-relevant metric. This means the output is unpredictable and cannot be reliably used for analysis that depends on ranking.

**To-Be (Proposed Enhancement):**
To make the report meaningful for analysis, the enhancement will modify the report to sort the results by **Net Value** in descending order *before* displaying them. This ensures that when a user asks for the 'top 10' items, they are seeing the 10 sales items with the highest Net Value that match their filter criteria. The final output will be presented in a modern **ALV Grid**, providing users with standard functions like filtering, sorting, and subtotals.

*Note on Scalability:* While this initial enhancement focuses on sorting by Net Value, the design should be scalable to accommodate additional sorting options in future iterations (e.g., sorting by Sales Document Date).

##### **2.2.1 Special Requirements**
N/A

#### **2.3 SAP Standard Reference**
This is a custom report (`Z_MCCAIN_SALES_ANALYSIS`) and does not modify any SAP standard transaction.

#### **2.4 Inputs and Dependencies**
*   **User Inputs:** The report's output is driven entirely by the filter criteria entered by the user on the selection screen.
*   **System Dependencies:** The report's functionality is dependent on the creation of a global data dictionary type, `ty_sales`. This is a technical prerequisite that must be in place before development work on the report can begin.

#### **2.5 Regulatory / Compliance Requirements**
N/A

#### **2.6 Assumptions**
*   Users have the necessary security authorizations to view sales data.
*   Users are familiar with the business context of Sales Documents and Material Numbers.

---

### **3. Enhancement Specifications**

#### **3.2 Transaction Screens**
The report has one initial screen for entering selection criteria.

**Selection Screen Fields:**
*   **Sales Document:** Allows the user to filter the report by one or more Sales Document Numbers. Users can enter single numbers, a range of numbers (e.g., 1000 to 2000), or exclude specific numbers.
*   **Material Number:** Allows the user to filter by one or more Material Numbers. This field has the same flexibility as the Sales Document filter (single values, ranges, exclusions).
*   **Number of Top Items to Display (P_TOP):** A single field where the user can specify the maximum number of records they want the report to return. The default value is 10.

#### **3.3 Enhancement Processing Logic – Key steps**
The report will execute in the following sequence:
1.  **Input:** The system presents the selection screen to the user.
2.  **Data Retrieval:** Based on the **Sales Document** and **Material Number** filters entered, the system queries the sales document item database (`VBAP`).
3.  **Check for Data:** The system checks if any records were returned from the database.
    *   If no data is found, the system will display a user-friendly message and stop processing (see Section 4: Error Handling).
    *   If data is found, the system proceeds to the next step.
4.  **Sorting:** The system sorts the retrieved data by **Net Value** in descending order.
5.  **Data Limiting:** The system restricts the sorted dataset to the number of records specified in the **Number of Top Items to Display** field.
6.  **Display:** The system displays the final, sorted, and limited data in an **ALV Grid**.

#### **3.4 Table Data Access**
*   **Primary Data Source:** The report retrieves all its data from the **Sales Document: Item Data** table (`VBAP`). This table contains the detailed information for each line item within a sales order.
*   **Required Fields:** To provide complete context, the report must retrieve and display the **Net Value** (`NETWR`) and the corresponding **Document Currency** (`WAERK`) for each line item.

---

### **4. Error Handling**
If no data is found for the selected criteria, the report must not produce a blank screen. Instead, it will display a clear, user-friendly message on the screen: **'No sales data found for the selected criteria.'** Standard SAP system error messages will apply for invalid data formats entered on the selection screen.

---

### **5. Level 0 Test Requirements**
The following test scenarios must be executed to validate the report's functionality:
*   **Scenario 1:** Run the report with a specific, known Sales Document Number. Verify that the correct items for that document are displayed.
*   **Scenario 2:** Run the report with a specific Material Number. Verify that the report displays sales items containing that material.
*   **Scenario 3:** Run the report using both a Sales Document Number and a Material Number filter.
*   **Scenario 4:** Change the 'Number of Top Items to Display' to 5. Verify that only 5 rows are displayed.
*   **Scenario 5:** Run the report with filter criteria that are known to return no results. Verify that the message 'No sales data found for the selected criteria' is displayed.
*   **Scenario 6 (Core Enhancement Test):** Given a selection that returns 20+ records, set 'Number of Top Items to Display' to 10. Verify the output displays exactly 10 records and that they are sorted by Net Value in descending order. Verify the Document Currency is displayed next to the Net Value for each record.

---

### **6. Additional Information**
N/A

---

### **7. Acronyms, Abbreviations, and Terms**

| Term/Acronym | Description                               |
| :----------- | :---------------------------------------- |
| **ALV**      | **SAP List Viewer.** A standard SAP tool for displaying data in a flexible grid format, which includes built-in user functions like interactive sorting, filtering, and subtotals. |
| **Sales Document** | The SAP document representing a sales transaction, such as a sales order. |
| **Item**     | A single line item within a Sales Document. |
| **Material Number** | The unique identifier for a product or service. |
| **Net Value**| The value of a sales document item after discounts and surcharges. |

---

### **8. References**
*   Technical Analysis Document: `ABAP Report Analysis: Z_MCCAIN_SALES_ANALYSIS`

---

### **9. Document Attributes**
| Attribute       | Value                                   |
| :-------------- | :-------------------------------------- |
| **Author**      | Business Systems Analyst                |
| **Status**      | Final                                   |
| **Creation Date** | [Date]                                  |
| **Version**     | 2.0                                     |

<br>
<hr>
<br>

# **Part 2: Technical Specification**

---

### **1. Enhancement Narrative**

#### **1.1 Request Details**
This document provides the technical specification for the custom ABAP report `Z_MCCAIN_SALES_ANALYSIS`. The purpose of this document is to provide developers with a complete and precise understanding of the report's internal workings to facilitate maintenance and future enhancements, incorporating mandatory changes from the latest functional specification.

#### **1.2 Enhancement Definition**
The Sales Analysis Report is a tool designed to provide a quick view of the top sales document items based on their net value. It allows users to filter sales data by Sales Document Number and Material Number. The key technical enhancements are:
1.  **Sorting:** The data selection is now sorted by Net Value (`NETWR`) in descending order to ensure the report displays the highest-value items.
2.  **Modernization:** The output is rendered using a modern ALV Grid (`cl_salv_table`) instead of the obsolete classical `WRITE` statement, providing a more feature-rich and user-friendly interface.

#### **1.3 Technical Overview**
`Z_MCCAIN_SALES_ANALYSIS` is an executable report program that performs a direct database query on the sales document item table (`VBAP`). It uses selection screen inputs to filter the data, sorts the results by net value, and limits the result set using the `UP TO n ROWS` clause. The output is displayed in an ALV Grid. The program is structured using subroutines (`PERFORM`).

---

### **2. Enhancement Functional Requirements**

#### **2.1 Business Requirements**
##### **2.1.1 Objective**
Enable business users to perform quick spot-checks on sales data by rapidly retrieving a small, filtered sample of the highest-value sales items.

##### **2.1.2 Scope**
*   **In-Scope:** Query and display data from the Sales Document Item table (`VBAP`). Filtering by Sales Document Number and Material Number. Sorting results by net value. Displaying results in an ALV Grid.
*   **Out-of-Scope:** Does not access or display header data (`VBAK`) or other master data. The report is display-only.

#### **2.2 As-Is/To-Be Functionality**
*   **As-Is:** The report retrieved an arbitrary set of records based on filter criteria. The `UP TO p_top ROWS` feature was unpredictable as there was no sorting logic. The output was a basic list.
*   **To-Be:** The data selection logic is modified to sort results by Net Value (`NETWR`) in descending order before limiting the number of records. The output is presented in a modern, interactive ALV Grid, which includes the document currency. This ensures the 'top' records shown are the ones with the highest value and provides a superior user experience.

#### **2.3 SAP Standard Reference**
This is a custom report and does not modify any SAP standard transaction.

#### **2.4 Inputs and Dependencies**
*   **User Inputs:** Selection criteria entered on the selection screen.
*   **System Dependencies:** The report's structure relies on a local data dictionary type (`ty_sales`) defined within the program.

#### **2.5 Regulatory / Compliance Requirements**
None.

#### **2.6 Assumptions**
*   Users have the necessary security authorizations to view `VBAP` data.
*   The system has the necessary components for `cl_salv_table` functionality.

---

### **3. Enhancement Specifications**

#### **3.1 Program Details**
*   **Program ID:** `Z_MCCAIN_SALES_ANALYSIS`
*   **Program Title:** Top Sales Items Analysis Report
*   **Program Type:** Executable Program (Report)

#### **3.2 Data Declarations**
##### **3.2.1 Type Definitions**
A local type `ty_sales` is required within the program. Its structure must be as follows:

```abap
TYPES: BEGIN OF ty_sales,
         vbeln TYPE vbap-vbeln,  " Sales Document
         posnr TYPE vbap-posnr,  " Sales Document Item
         matnr TYPE vbap-matnr,  " Material Number
         netwr TYPE vbap-netwr,  " Net Value in Document Currency
         waerk TYPE vbap-waerk,  " Document Currency
       END OF ty_sales.
```

##### **3.2.2 Internal Tables and Structures**
| Variable Name   | Type                 | Description                                                          |
| :-------------- | :------------------- | :------------------------------------------------------------------- |
| `gt_sales_data` | `TABLE OF ty_sales`  | Global internal table to hold the sales data retrieved from `VBAP`.  |
| `go_alv`        | `REF TO cl_salv_table` | Global object reference for the ALV Grid instance.                   |

#### **3.3 Selection Screen Objects (Dynpro 1000)**
The following objects are defined for user input:

| Object Type      | Technical Name | `FOR` / `TYPE` Reference | `DEFAULT` Value | Text / Description                  |
| :--------------- | :------------- | :----------------------- | :-------------- | :---------------------------------- |
| `SELECT-OPTIONS` | `s_vbeln`      | `vbak-vbeln`             | N/A             | Sales Document Number               |
| `SELECT-OPTIONS` | `s_matnr`      | `mara-matnr`             | N/A             | Material Number                     |
| `PARAMETER`      | `p_top`        | `i` (Integer)            | `10`            | Number of Top Items to Display      |

#### **3.4 Detailed Logic Flow**
The program execution is controlled by events and subroutines.

##### **3.4.1. START-OF-SELECTION Event**
This is the main processing block that drives the report's execution.
1.  **`PERFORM get_sales_data.`**: Calls the data retrieval subroutine.
2.  **`PERFORM display_alv_report.`**: Calls the data display subroutine.

##### **3.4.2. Subroutine: `get_sales_data`**
This subroutine handles all database interaction.
*   **Logic:** Executes a single `SELECT` statement to fetch the top N sales items from the `VBAP` table, sorted by net value.
*   **ABAP Code Snippet:**
    ```abap
    FORM get_sales_data.
      SELECT vbeln, posnr, matnr, netwr, waerk
        FROM vbap
        INTO CORRESPONDING FIELDS OF TABLE gt_sales_data
        WHERE vbeln IN s_vbeln AND matnr IN s_matnr
        ORDER BY netwr DESCENDING
        UP TO p_top ROWS.
    ENDFORM.
    ```
*   **Key Logic Points:**
    *   **Fields Selected:** `vbeln`, `posnr`, `matnr`, `netwr`, and `waerk`.
    *   **Target:** Data is inserted into the `gt_sales_data` internal table.
    *   **Filtering:** Filters on sales document number (`vbeln`) and material number (`matnr`).
    *   **Sorting:** `ORDER BY netwr DESCENDING` ensures the database sorts the records by the highest net value first.
    *   **Limitation:** `UP TO p_top ROWS` clause is applied *after* sorting to restrict the result set to the true top N records.

##### **3.4.3. Subroutine: `display_alv_report`**
This subroutine handles the presentation of the final data using the ALV Grid.
*   **Logic:** Checks if data was selected. If so, it creates and displays an instance of `cl_salv_table`.
*   **ABAP Code Snippet:**
    ```abap
    FORM display_alv_report.
      IF gt_sales_data IS INITIAL.
        MESSAGE 'No sales data found for the selected criteria' TYPE 'I'.
        LEAVE LIST-PROCESSING.
      ENDIF.

      TRY.
          cl_salv_table=>factory(
            IMPORTING
              r_salv_table   = go_alv
            CHANGING
              t_table        = gt_sales_data ).
        CATCH cx_salv_msg.
          " Add appropriate error handling for ALV creation failure
          MESSAGE 'Error creating ALV Grid display.' TYPE 'E'.
          LEAVE LIST-PROCESSING.
      ENDTRY.

      go_alv->display( ).
    ENDFORM.
    ```

#### **3.5 Output Details**
*   **Output Method:** ALV Grid, instantiated from the `cl_salv_table` class. This provides standard ALV features like sorting, filtering, and exporting.
*   **Output Format:** A grid where each row represents a sales document item.
*   **Output Columns (default order):**
    1.  Sales Document (`vbeln`)
    2.  Item Number (`posnr`)
    3.  Material Number (`matnr`)
    4.  Net Value (`netwr`)
    5.  Document Currency (`waerk`)

#### **3.6 Table Data Access**
*   **Primary Table:** `VBAP` (Sales Document: Item Data)
*   **Access Type:** Read-only (`SELECT`).
*   **Fields Read:** `vbeln`, `posnr`, `matnr`, `netwr`, `waerk`.

---

### **4. Error Handling**
*   **No Data Found:** After the `SELECT` statement, the program checks if the internal table `gt_sales_data` is initial. If it is, an informational message `'No sales data found for the selected criteria'` is displayed to the user, and the program exits gracefully.
*   **ALV Creation Failure:** A `TRY...CATCH` block is used around the `cl_salv_table=>factory` call to handle any exceptions during ALV instantiation. An error message will be displayed if creation fails.
*   **Standard Errors:** Standard SAP error handling applies for invalid input on the selection screen.

---

### **5. Level 0 Test Requirements**
*   **Scenario 1 (Core Logic Validation):**
    *   **Action:** Execute the report with `p_top` set to '5' and with selection criteria known to return more than 5 records.
    *   **Expected Result:** The ALV grid displays exactly 5 rows. The rows are sorted in descending order based on the 'Net Value' column. The 'Document Currency' column is visible and populated.
*   **Scenario 2:** Execute with a known `s_vbeln`. Verify correct items are displayed.
*   **Scenario 3:** Execute with a known `s_matnr`. Verify correct items are displayed.
*   **Scenario 4 (No Data):** Execute with criteria known to return no results.
    *   **Expected Result:** An informational message "No sales data found for the selected criteria" is displayed on the status bar. No ALV grid is shown.

---

### **6. Code Quality and Improvement Suggestions**
*   **Mandatory Implementation:** The core logic has been updated as per the functional specification. This includes:
    *   The `SELECT` statement now correctly uses `ORDER BY netwr DESCENDING` before the `UP TO p_top ROWS` clause.
    *   The output has been migrated from `WRITE` statements to the `cl_salv_table` ALV Grid.
*   **Best Practice Recommendations for Future Maintenance:**
    *   Refactor `PERFORM...ENDFORM` subroutines into methods of a local class to improve structure, encapsulation, and testability.
    *   Replace obsolete `TABLES` statements (if any remain) with explicit `DATA` work area declarations referenced by the `SELECT-OPTIONS`.
    *   Utilize modern ABAP syntax where appropriate, such as inline declarations (e.g., `DATA(lo_alv)`).

---

### **7. Acronyms, Abbreviations, and Terms**

| Term/Acronym | Description                                                      |
| :------------- | :--------------------------------------------------------------- |
| **ALV**        | SAP List Viewer, a standard tool for displaying data in a grid. |
| **VBAP**       | The underlying database table for Sales Document Item Data.      |
| **Dynpro**     | Dynamic Program, the SAP term for a screen.                      |
| **TS**         | Technical Specification.                                         |

---

### **8. References**
*   Functional Specification: `Sales Analysis Report, v2.0`
*   ABAP Code Analysis: `ABAP Report Analysis: Z_MCCAIN_SALES_ANALYSIS`
*   Output Reviewer Feedback Memo

---

### **9. Document Attributes**

| Attribute       | Value                           |
| :-------------- | :------------------------------ |
| **Author**      | Technical Documentation Specialist |
| **Status**      | Final (Revised)                 |
| **Creation Date** | [Date of Revision]              |
| **Version**     | 2.0                             |